<a href="https://colab.research.google.com/github/abhi147p/Abhishek_INFO5731_Spring2023/blob/main/In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [19]:
# Write your code here

# getting the data
# You code here (Please add comments in the code):

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

def get_soup(url):
    """
        params1: url (contains the url of google scholar page)
        return: soup (fetching the url page data and then further converted to html parser)
    """
  # headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    headers = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36')}
               # 'X-User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 FKUA/website/42/website/Desktop')}
    try:
        # requesting for data using requests with url and headers for authentication
        data = requests.get(url, headers=headers)
        # print(f"Extracted the data with response as {data.status_code}")
        if data.status_code != 200:
            raise Exception(f"Failed to fetch data: {data.status_code} and text as {data.text}")
    except Exception as ex:
        print(f"Exception occurred as {ex}")
        return None
    soup = BeautifulSoup(data.text, "html.parser")
    return soup

def fetch_data_web(pages):

  final_data = []
  for i in range(1, pages+1):
    url = f"https://www.flipkart.com/search?q=Watches&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={i}"
    soup = get_soup(url)
    # print(soup.prettify())
    products_data = soup.findAll("div", attrs={"class": "_1AtVbE col-12-12"})
    for prod_data in products_data:
      sub_prod_data = prod_data.findAll("div", attrs={"class": "_1xHGtK _373qXS"})
      for data in sub_prod_data:
        prod_url = "https://www.flipkart.com" + data.find("a", attrs={"class": "_2UzuFa"}).get("href")
        brand = data.find("div", attrs={"class": "_2WkVRV"}).text.strip()
        title = data.find("a", attrs={"class": "IRpwTa"}).get("title")
        new_price = data.find("div", attrs={"class": "_30jeq3"})
        old_price = data.find("div", attrs={"class": "_3I9_wc"})
        if new_price:
          new_price = new_price.text.strip()
        else:
          new_price = "No"
        if old_price:
          old_price = old_price.text.strip()
        else:
          old_price = "No"
        # discount = data.find("div", attrs={"class": "_3Ay6Sb"}).text.strip()
        # free_delivery = data.find("div", attrs={"class": "_2Tpdn3"})
        # if free_delivery:
        #   free_delivery = free_delivery.text.strip()
        # else:
        #   free_delivery = "No free Delivery"
        soup_data = get_soup(prod_url)
        # print(prod_url)
        # print(soup_data)
        # break
        # rating = "0"
        # if soup_data:
        if soup_data:
          rating = soup_data.find("div", attrs={"class": "_3LWZlK _3uSWvT"})
          if rating:
            rating = rating.text
          else:
            rating = "No rating found"
          reviews = soup_data.findAll("div", attrs={"class": "_6K-7Co"})
          for x in reviews:
            final_data.append([title, prod_url, brand, new_price, old_price, rating, x.text])
        else:
          rating = "No rating reviews found"
        # final_data.append([title, prod_url, brand, new_price, old_price, rating, review])
      time.sleep(1)
    print(f"Processed {i} pages")
  
  df = pd.DataFrame(final_data, columns=["Title", "url", "brand", "new_price", "old_price", "rating", "review"])
  return df

df = fetch_data_web(int(input("Enter no of pages: ")))
df.head()

Enter no of pages: 3
Processed 1 pages
Processed 2 pages
Processed 3 pages


,Title,url,brand,new_price,old_price,rating,review
0,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,₹291,₹999,3.9,Very nice watch ⌚ delivered by Flipkart. Looks...
1,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,₹291,₹999,3.9,Super
2,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,₹291,₹999,3.9,"It's ossamm product, it's Value for money"
3,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,hmte,₹299,₹999,3.9,Very nice watch ⌚ delivered by Flipkart. Looks...
4,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,hmte,₹299,₹999,3.9,Super


In [23]:
df = df[df['old_price'] != "No"]

In [26]:
# converting str to numeric

df['new_price'] = df['new_price'].str.replace(',', '').str[1:]
df['old_price'] = df['old_price'].str.replace(',', '').str[1:]

In [27]:
df['new_price'] = df['new_price'].astype(int)
df['old_price'] = df['old_price'].astype(int)
df['rating'] = df['rating'].astype(float)

df.head()

,Title,url,brand,new_price,old_price,rating,review
0,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,291,999,3.9,Very nice watch ⌚ delivered by Flipkart. Looks...
1,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,291,999,3.9,Super
2,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,hmte,291,999,3.9,"It's ossamm product, it's Value for money"
3,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,hmte,299,999,3.9,Very nice watch ⌚ delivered by Flipkart. Looks...
4,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,hmte,299,999,3.9,Super


In [49]:
df = df[['Title', 'url', 'review']]
df.head()

,Title,url,review
0,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Very nice watch ⌚ delivered by Flipkart. Looks...
1,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Super
2,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,"It's ossamm product, it's Value for money"
3,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Very nice watch ⌚ delivered by Flipkart. Looks...
4,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Super


In [50]:
df['review'].isna().sum()

0

In [51]:
import nltk, re, string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
df.head()

,Title,url,review
0,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Very nice watch ⌚ delivered by Flipkart. Looks...
1,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Super
2,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,"It's ossamm product, it's Value for money"
3,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Very nice watch ⌚ delivered by Flipkart. Looks...
4,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Super


In [58]:
import re

df['clean_Data'] = df['review'].map(lambda x: re.sub('[,\.!?]', '', x))
df['clean_Data'] = df['clean_Data'].map(lambda x: x.lower())
df.head()

,Title,url,review,clean_Data
0,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Very nice watch ⌚ delivered by Flipkart. Looks...,very nice watch ⌚ delivered by flipkart looks ...
1,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,Super,super
2,Day&Date Series Analog Watch - For Boys HM-3172,https://www.flipkart.com/hmte-day-date-series-...,"It's ossamm product, it's Value for money",it's ossamm product it's value for money
3,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Very nice watch ⌚ delivered by Flipkart. Looks...,very nice watch ⌚ delivered by flipkart looks ...
4,Analog Watch - For Boys,https://www.flipkart.com/hmte-analog-watch-boy...,Super,super


In [65]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [90]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

review_data = df['review'].values.tolist()
review_data_words = list(sent_to_words(review_data))
# remove stop words
review_data_words = remove_stopwords(review_data_words)
print(review_data_words[:1])

[['nice', 'watch', 'delivered', 'flipkart', 'looks', 'nice', 'like', 'premium', 'watch', 'hand', 'feel', 'nice', 'fitting', 'good', 'really', 'happy', 'watch', 'thanks', 'flipkart', 'better', 'services', 'fast', 'delivery']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
bigram = gensim.models.Phrases(review_data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[review_data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

review_data_words_bigrams = make_bigrams(review_data_words)

# Do lemmatization keeping only noun, adj, vb, adv
review_data_lemmatized = lemmatization(review_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(review_data_lemmatized[:1])

[['nice', 'watch', 'deliver', 'flipkart', 'look', 'nice', 'premium', 'watch', 'hand', 'feel', 'nice', 'fitting', 'good', 'really', 'happy', 'watch', 'thank', 'flipkart', 'well', 'service', 'fast', 'delivery']]


In [92]:
review_id2word = corpora.Dictionary(review_data_lemmatized)
review_texts = review_data_lemmatized
review_corpus = [review_id2word.doc2bow(text) for text in review_texts]
print(review_corpus[:1])
[[(review_id2word[i], f) for i, f in c] for c in review_corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1)]]


[[('deliver', 1),
  ('delivery', 1),
  ('fast', 1),
  ('feel', 1),
  ('fitting', 1),
  ('flipkart', 2),
  ('good', 1),
  ('hand', 1),
  ('happy', 1),
  ('look', 1),
  ('nice', 3),
  ('premium', 1),
  ('really', 1),
  ('service', 1),
  ('thank', 1),
  ('watch', 3),
  ('well', 1)]]

In [93]:
review_id2word[0]

'deliver'

In [94]:
# Sample model for lda generating total of 20 topics
from pprint import pprint
lda_model = gensim.models.ldamodel.LdaModel(corpus=review_corpus,
                                           id2word=review_id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[review_corpus]

[(0,
  '0.061*"packing" + 0.002*"pretty" + 0.002*"money" + 0.002*"value" + '
  '0.002*"watch" + 0.002*"also" + 0.002*"good" + 0.002*"go" + 0.002*"thank" + '
  '0.002*"nice"'),
 (1,
  '0.311*"good" + 0.137*"quality" + 0.096*"love" + 0.066*"price" + '
  '0.052*"also" + 0.048*"product" + 0.045*"really" + 0.044*"look" + '
  '0.043*"design" + 0.031*"well"'),
 (2,
  '0.235*"much" + 0.223*"thank" + 0.193*"love" + 0.153*"flipkart" + '
  '0.022*"really" + 0.000*"lot" + 0.000*"hmmm" + 0.000*"thi" + 0.000*"jab" + '
  '0.000*"ayi"'),
 (3,
  '0.180*"watch" + 0.133*"nice" + 0.098*"time" + 0.080*"month" + '
  '0.052*"product" + 0.051*"quality" + 0.047*"also" + 0.039*"date" + '
  '0.034*"aaftab" + 0.034*"proof"'),
 (4,
  '0.002*"people" + 0.002*"question" + 0.002*"friendly" + 0.002*"hager" + '
  '0.002*"magical" + 0.002*"notice" + 0.002*"eddy" + 0.002*"number" + '
  '0.002*"try" + 0.002*"collection"'),
 (5,
  '0.123*"watch" + 0.120*"happy" + 0.081*"amazing" + 0.068*"go" + 0.060*"work" '
  '+ 0.054*"gi

In [96]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=review_data_lemmatized, dictionary=review_id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45357167129272263


In [87]:
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [101]:
# calculating model coherence value for various number of topics to get the best k number
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
start, limit, step=2, 20, 2
model_list, coherence_values = compute_coherence_values(dictionary=review_id2word, corpus=review_corpus, texts=review_data_lemmatized, start=start, limit=limit, step=step)
coherence_values_all = []
for m, cv in zip(range(start, limit, step), coherence_values):
    coherence_values_all.append(round(cv,4))
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4827
Num Topics = 4  has Coherence Value of 0.4239
Num Topics = 6  has Coherence Value of 0.4313
Num Topics = 8  has Coherence Value of 0.3949
Num Topics = 10  has Coherence Value of 0.3498
Num Topics = 12  has Coherence Value of 0.3585
Num Topics = 14  has Coherence Value of 0.4072
Num Topics = 16  has Coherence Value of 0.3636
Num Topics = 18  has Coherence Value of 0.4574


In [102]:
sum(coherence_values_all)/len(coherence_values_all)

0.40769999999999995

As we can see the values are first decreased and then increasing. Hence taking the value, which is near to mean of all coherence values. Taking 8 topics with value 0.39. 

In [103]:
optimal_model = model_list[3] # getting the topics=8 model, which is in 3rd index of our model and getting the 8 topics.
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.149*"money" + 0.119*"value" + 0.042*"cool" + 0.040*"purchase" + '
  '0.035*"simple" + 0.033*"recommend" + 0.028*"go" + 0.026*"satisfied" + '
  '0.025*"want" + 0.022*"royal"'),
 (1,
  '0.153*"good" + 0.139*"watch" + 0.094*"product" + 0.085*"nice" + '
  '0.055*"quality" + 0.036*"look" + 0.026*"price" + 0.025*"love" + 0.024*"buy" '
  '+ 0.022*"design"'),
 (2,
  '0.075*"thank" + 0.066*"amazing" + 0.053*"much" + 0.053*"picture" + '
  '0.052*"flipkart" + 0.041*"use" + 0.039*"product" + 0.036*"receive" + '
  '0.033*"love" + 0.030*"order"'),
 (3,
  '0.069*"nice" + 0.063*"watch" + 0.061*"product" + 0.060*"time" + '
  '0.049*"month" + 0.049*"much" + 0.037*"give" + 0.027*"also" + 0.024*"date" + '
  '0.024*"big"'),
 (4,
  '0.085*"excellent" + 0.045*"expect" + 0.040*"strap" + 0.034*"trust" + '
  '0.034*"quality" + 0.033*"watch" + 0.032*"look" + 0.027*"give" + '
  '0.026*"super" + 0.025*"big"'),
 (5,
  '0.060*"order" + 0.049*"think" + 0.038*"look" + 0.036*"go" + 0.036*"guy" + '
  '0.034*"i

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here






## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here





## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here





## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



